## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
enlace

'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210331.ods'

In [8]:
try:
    datos_row = pd.read_excel(enlace,sheet_name='Comunicación')
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210331.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [9]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,1098720,168000,381600,1648320,1441414,0.874475,507430,2021-03-30
1,Aragón,209745,35300,59900,304945,250339,0.820932,89278,2021-03-30
2,Asturias,203315,32700,46000,282015,225915,0.801074,87093,2021-03-30
3,Baleares,112680,18000,52700,183380,160884,0.877326,45509,2021-03-30
4,Canarias,232800,35800,98100,366700,304229,0.829640,104712,2021-03-30
5,Cantabria,97845,15300,26200,139345,114080,0.818687,44056,2021-03-30
6,Castilla y Leon,463035,78300,108100,649435,534134,0.822460,198064,2021-03-30
7,Castilla La Mancha,300555,49600,92200,442355,356085,0.804976,122854,2021-03-30
8,Cataluña,1027880,168400,350800,1547080,1274944,0.824097,421043,2021-03-30
9,C. Valenciana,586360,96800,228100,911260,768217,0.843027,251514,2021-03-30


..hacemos una copia para hacerle las transformaciones necesarias:

In [10]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [11]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [12]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [13]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [14]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [15]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,1098720,168000,381600,1648320,1441414,0.874475,507430,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,209745,35300,59900,304945,250339,0.820932,89278,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,203315,32700,46000,282015,225915,0.801074,87093,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,112680,18000,52700,183380,160884,0.877326,45509,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,232800,35800,98100,366700,304229,0.829640,104712,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,97845,15300,26200,139345,114080,0.818687,44056,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,463035,78300,108100,649435,534134,0.822460,198064,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,300555,49600,92200,442355,356085,0.804976,122854,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,1027880,168400,350800,1547080,1274944,0.824097,421043,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,586360,96800,228100,911260,768217,0.843027,251514,30/3/21,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [16]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [17]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [18]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla',
       'Fuerzas Armadas', 'Totales'], dtype=object)

In [19]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,31/3/21,Andalucía,1098720,168000,381600,1648320,1441414,0.874475,507430,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,209745,35300,59900,304945,250339,0.820932,89278,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,203315,32700,46000,282015,225915,0.801074,87093,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,112680,18000,52700,183380,160884,0.877326,45509,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,232800,35800,98100,366700,304229,0.829640,104712,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,97845,15300,26200,139345,114080,0.818687,44056,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,463035,78300,108100,649435,534134,0.822460,198064,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla La Mancha,300555,49600,92200,442355,356085,0.804976,122854,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,1027880,168400,350800,1547080,1274944,0.824097,421043,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,586360,96800,228100,911260,768217,0.843027,251514,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [20]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [21]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [22]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,30/3/21,País Vasco,275940.0,30800.0,94900.0,401640,361382,0.899766,101299.0,NaN,29/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1201,30/3/21,Ceuta,6660.0,900.0,3700.0,11260,11142,0.989520,3600.0,NaN,29/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1202,30/3/21,Melilla,6660.0,800.0,3700.0,11160,10297,0.922670,2419.0,NaN,29/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1203,30/3/21,Fuerzas Armadas,19500.0,0.0,33100.0,52600,28065,0.533555,4848.0,NaN,29/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [23]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [24]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [25]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [26]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,31/3/21,País Vasco,314550.0,52900.0,100200.0,467650,370971,0.793266,105534.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1222,31/3/21,Ceuta,7830.0,1300.0,3900.0,13030,11226,0.861550,3601.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1223,31/3/21,Melilla,7830.0,1200.0,3900.0,12930,10413,0.805336,2423.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1224,31/3/21,Fuerzas Armadas,19500.0,0.0,36100.0,55600,37606,0.676367,4991.0,NaN,30/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [27]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales',
       'Fuerzas Armadas'], dtype=object)

y volvemos a subirlo..

In [28]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

# Institucionalizados

In [110]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_institucionalizados_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Institucionalizados'

In [111]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [112]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210331.ods


In [113]:
datos_row

,Unnamed: 0,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA
0,Andalucía,41651,40770,38848,0.978848,0.932703
1,Aragón,17829,17829,17025,1.000000,0.954905
2,Asturias,11948,11577,11115,0.968949,0.930281
3,Baleares,5496,5117,4869,0.931041,0.885917
4,Canarias,10531,10531,9441,1.000000,0.896496
5,Cantabria,6919,6821,6429,0.985836,0.929181
6,Castilla y Leon,45800,45717,43930,0.998188,0.959170
7,Castilla - La Mancha,25566,24387,22610,0.953884,0.884378
8,Cataluña,73343,73068,62908,0.996250,0.857723
9,C. Valenciana,29500,29240,26788,0.991186,0.908068


In [114]:
datos_hoy = datos_row.copy()

In [115]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [116]:
datos_hoy.columns = ['ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA', 'date_pub',
       'source_name', 'source']

In [117]:
datos_hoy

,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,date_pub,source_name,source
0,Andalucía,41651,40770,38848,0.978848,0.932703,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,17829,17829,17025,1.000000,0.954905,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,11948,11577,11115,0.968949,0.930281,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,5496,5117,4869,0.931041,0.885917,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,10531,10531,9441,1.000000,0.896496,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,6919,6821,6429,0.985836,0.929181,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,45800,45717,43930,0.998188,0.959170,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla - La Mancha,25566,24387,22610,0.953884,0.884378,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,73343,73068,62908,0.996250,0.857723,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,29500,29240,26788,0.991186,0.908068,31/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [118]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA',
       'source_name', 'source']]

In [119]:
datos_hoy

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,6919,6821,6429,0.985836,0.929181,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,45800,45717,43930,0.998188,0.959170,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla - La Mancha,25566,24387,22610,0.953884,0.884378,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,73343,73068,62908,0.996250,0.857723,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,29500,29240,26788,0.991186,0.908068,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [120]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_20210331'

In [121]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

In [122]:
datos_hoy.to_csv(nombre_fichero_acumulado+'.csv',index = False)

# Etarios 1 dosis

In [124]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_1dosis_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_al_menos_1_dosis'

In [125]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [126]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210331.ods


In [127]:
datos_hoy = datos_row.copy()

In [128]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [129]:
datos_hoy.columns = ['ccaa', 'Personas con al menos 1 dosis ≥80 años',
       'Población INE≥80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [130]:
datos_hoy = datos_hoy [['date_pub','ccaa', 'Personas con al menos 1 dosis ≥80 años',
       'Población INE≥80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 
       'source_name', 'source']]

In [131]:
datos_hoy

,date_pub,ccaa,Personas con al menos 1 dosis ≥80 años,Población INE≥80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.907781,21914,650381,0.033694,60757,916147,...,625846,0.039920,144,181921,0.000792,933984,7062213,0.132251,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.721163,3553,118955,0.029868,11417,156367,...,89180,0.061595,28,25106,0.001115,161061,1132764,0.142184,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293,4258,109249,0.038975,15803,149652,...,54179,0.051976,30,15563,0.001928,138822,901209,0.154040,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.668948,2346,80516,0.029137,15295,117231,...,85839,0.040972,31,23300,0.001330,115375,986279,0.116980,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600869,15913,154531,0.102976,33435,235092,...,163899,0.034125,27,44352,0.000609,199517,1871033,0.106635,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,30858,41042,0.751864,1827,53066,0.034429,4962,77345,...,34935,0.040332,5,10791,0.000463,70024,501384,0.139661,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,158345,223657,0.707981,12861,245410,0.052406,26304,318575,...,144422,0.048808,89,40684,0.002188,336070,2092873,0.160578,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla - La Mancha,99503,135194,0.736002,6609,159101,0.041540,17253,218444,...,149518,0.037012,91,42364,0.002148,233231,1719964,0.135602,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,240692,447196,0.538225,36372,635292,0.057252,142711,837584,...,556316,0.059851,270,159864,0.001689,853901,6531658,0.130733,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,212048,276531,0.766815,11809,437862,0.026970,33846,580728,...,352834,0.040367,106,103049,0.001029,516703,4269305,0.121027,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [132]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_1dosis_20210331'

In [133]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

In [134]:
datos_hoy.to_csv(nombre_fichero_acumulado+'.csv',index = False)

# Etarios con pauta completa

In [135]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_pauta_completa_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_pauta_completa'

In [136]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [137]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210331.ods


In [138]:
datos_hoy = datos_row.copy()

In [139]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [140]:
datos_hoy.columns = ['ccaa', 'Personas pauta completa ≥80 años',
       'Población INE≥80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [141]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Personas pauta completa ≥80 años',
       'Población INE≥80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE',
       'source_name', 'source']]

In [142]:
datos_hoy

,date_pub,ccaa,Personas pauta completa ≥80 años,Población INE≥80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,240454,413516,0.581487,12788,650381,0.019662,41807,916147,...,625846,0.018677,83,181921,0.000456,507430,7062213,0.071851,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,40245,98807,0.407309,3048,118955,0.025623,9273,156367,...,89180,0.015934,25,25106,0.000996,89278,1132764,0.078814,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,37353,87826,0.425307,3490,109249,0.031945,9709,149652,...,54179,0.042636,24,15563,0.001542,87093,901209,0.096640,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,12403,49542,0.250353,2088,80516,0.025933,4856,117231,...,85839,0.014318,30,23300,0.001288,45509,986279,0.046142,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,31183,90236,0.345572,11216,154531,0.072581,9016,235092,...,163899,0.014332,19,44352,0.000428,104712,1871033,0.055965,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,31/3/21,Cantabria,22177,41042,0.540349,1660,53066,0.031282,3704,77345,...,34935,0.017003,3,10791,0.000278,44056,501384,0.087869,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,31/3/21,Castilla y Leon,96294,223657,0.430543,8627,245410,0.035153,19766,318575,...,144422,0.017740,76,40684,0.001868,198064,2092873,0.094637,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,31/3/21,Castilla - La Mancha,46971,135194,0.347434,5152,159101,0.032382,12955,218444,...,149518,0.021609,74,42364,0.001747,122854,1719964,0.071428,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,31/3/21,Cataluña,145808,447196,0.326049,24886,635292,0.039173,45494,837584,...,556316,0.027939,188,159864,0.001176,421043,6531658,0.064462,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,31/3/21,C. Valenciana,114569,276531,0.414308,9437,437862,0.021552,22540,580728,...,352834,0.014338,83,103049,0.000805,251514,4269305,0.058912,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [143]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_20210331'

In [144]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

In [145]:
datos_hoy.to_csv(nombre_fichero_acumulado+'.csv',index = False)